In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
options_df = pd.read_csv('Banknifty_Options_2018-July_2023.csv')
options_df.head()

In [ ]:
spot_df = pd.read_csv('Banknifty_Spot_2018-2023.csv')
spot_df.head()

In [ ]:
def calculate_atm_strike_price_for_date(spot_df_2018, selected_date,entry_time):
    """
    This function calculates the ATM strike price for a given date at 9:30 am.

    Parameters:
    spot_df (pd.DataFrame): Input DataFrame with 'Date', 'Time', and 'Close' columns
    selected_date (str): The date for which to calculate the ATM strike price (in 'YYYY-MM-DD' format)
    time (str): The time for which to calculate the ATM strike price (in 'HH:MM:SS' format)

    Returns:
    float or None: The ATM strike price or None if the date/time is not found
    """
    
    # Ensure 'Time' column is in datetime format
    spot_df['Time'] = pd.to_datetime(spot_df['Time'], format='%H:%M:%S').dt.time
    spot_df['Date'] = pd.to_datetime(spot_df['Date'], format='%Y-%m-%d')

    # Filter data for the specified date and 10:29:59 am
    trade_time = pd.to_datetime(entry_time, format='%H:%M:%S').time()
    filtered_df = spot_df[(spot_df['Date'] == pd.to_datetime(selected_date, format='%Y-%m-%d')) &
                             (spot_df['Time'] == trade_time)]
    
    # Check if we have at least one row after filtering
    if not filtered_df.empty:
        # Calculate the average close price and determine the ATM strike price
        avg_close_price = filtered_df['Close'].mean()
        atm_strike_price = round(avg_close_price / 100) * 100
        return atm_strike_price
    else:
        # Return None if no rows are found for the specified date and time
        return None

In [335]:
calculate_atm_strike_price_for_date(spot_df, pd.to_datetime('2018-01-01', format='%Y-%m-%d'),'09:30:00')


25600

In [ ]:
def put_log_trades(trade_log, entry_date, expiry_date, entry_time, atm_strike_price, put_options_atm, put_options_otm,put_options_itm,capital, stop_loss,total_entry_premium, target):
    """
    This function logs trades by appending the options data to the trade log.

    Parameters:
    trade_log (list): List to store the trade log entries
    entry_date (str): The date at which the trade was entered
    expiry_date (str): The expiry date of the options
    entry_time (str): The time at which the trade was entered
    atm_strike_price (float): The ATM strike price for the selected date
    put_options_atm (pd.Series): Series containing the ATM put options data
    put_options_itm (pd.Series): Series containing the ITM put options data
    put_options_otm (pd.Series): Series containing the OTM put options data
    capital (float): The total capital available for trading
    stop_loss (float): Calculated stop loss
    target (float): Calculated target
    
   

    Returns:
    list: Updated trade log
    """
    trade_log.append({
        'Serial_No': len(trade_log) + 1,
        'Date': entry_date,
        'Expiry': expiry_date,
        'En_Date': entry_date,
        'Pe_En_Date': entry_date,
        'Ce_Ex_Date': None,  # Placeholder for call exit date
        'Pe_Ex_Date': None,  # Placeholder for put exit date
        'Atm_Strike': atm_strike_price,
        'Pe_Short_Strike': put_options_atm['Strike Price'],
        'Pe_otm_Long_Strike': put_options_otm['Strike Price'],
        'Pe_itm_Long_Strike': put_options_itm['Strike Price'],
        'Capital': capital,
        'Pe_Short_En_Price': put_options_atm['Close'],
        'Pe_Short_Ex_Price': None,  # Placeholder for put short exit price
        'Pe_otm_Long_En_Price': put_options_otm['Close'],
        'Pe_otm_Long_Ex_Price': None,  # Placeholder for put long exit price
        'Pe_itm_Long_En_Price': put_options_itm['Close'],
        'Pe_itm_Long_Ex_Price': None,  # Placeholder for put long exit price
        'Entry_Time': entry_time,
        'Exit Time': None,  # Placeholder for exit time
        'Current Market Price':None, #Have to take current market price from spot data
        'Ticker_Pe_Short': put_options_atm['Ticker'],
        'Ticker_otm_Pe_Long': put_options_otm['Ticker'],
        'Ticker_itm_Pe_Long': put_options_itm['Ticker'],
        'Exit_Reason': None,  # Placeholder for exit reason
        'Stop_Loss': stop_loss,
        'Total_Premium_Received':  total_entry_premium,
        'Target': target,
        'Total_PnL': None  # Placeholder for total P&L
    })
    return trade_log


In [ ]:
def filter_options_data(trade_log_df, options_df):
    """
    This function filters the options data for each trade in the trade log and combines them into a single DataFrame.
    """
    filtered_data = []
    for _, trade in trade_log_df.iterrows():
        
        pe_short_data = options_df[(options_df['Strike Price'] == trade['Pe_Short_Strike']) & 
                                   (options_df['ExpiryDate'] == trade['Expiry']) & 
                                   (options_df['Instrument Type'] == 'PE') & 
                                   (options_df['Date'] >= pd.to_datetime(trade['Date']))]
        
        pe_otm_long_data = options_df[(options_df['Strike Price'] == trade['Pe_otm_Long_Strike']) & 
                                  (options_df['ExpiryDate'] == trade['Expiry']) & 
                                  (options_df['Instrument Type'] == 'PE') & 
                                  (options_df['Date'] >= pd.to_datetime(trade['Date']))]
        
        pe_itm_long_data = options_df[(options_df['Strike Price'] == trade['Pe_itm_Long_Strike']) & 
                                  (options_df['ExpiryDate'] == trade['Expiry']) & 
                                  (options_df['Instrument Type'] == 'PE') & 
                                  (options_df['Date'] >= pd.to_datetime(trade['Date']))]
        
        print(pe_short_data)
        
        combined_data = pd.concat([pe_short_data, pe_otm_long_data,pe_itm_long_data]).sort_values(by=['Date', 'Time'])
        filtered_data.append(combined_data)
        
    return pd.concat(filtered_data).reset_index(drop=True)


In [336]:
def get_option_market_price(options_df, datetime, strike_price, expiry_date):
    """
    This function gets the current market price of the given ticker.
    """
    current_price = options_df[(options_df['Datetime'] == datetime) & (options_df['Strike Price'] == strike_price) & (options_df['ExpiryDate'] == expiry_date)]['Close']
    return current_price


In [337]:
options_df_2018[(options_df_2018['Datetime'] == pd.to_datetime('2018-01-04 09:30:00')) & (options_df_2018['Strike Price'] == trade_log_df['Pe_Short_Strike'].values[0]) & (options_df_2018['ExpiryDate'] == trade_log_df['Expiry'].values[0])]['Close']

59143    79.0
Name: Close, dtype: float64

In [316]:
trade_log_df['Entry_Time'].head()

0    09:30:00
0    09:30:00
0    09:30:00
0    09:30:00
0    09:30:00
Name: Entry_Time, dtype: object

In [310]:


def calculate_pnl(trade_log_df, options_df, datetime):
    atm_short_current_price = get_option_market_price(options_df_2018, datetime, trade_log_df['Pe_Short_Strike'].values[0], trade_log_df['Expiry'].values[0])
    print(atm_short_current_price)
    otm_long_current_price = get_option_market_price(options_df_2018, datetime, trade_log_df['Pe_otm_Long_Strike'].values[0], trade_log_df['Expiry'].values[0])
    print(otm_long_current_price)
    itm_long_current_price = get_option_market_price(options_df_2018, datetime, trade_log_df['Pe_itm_Long_Strike'].values[0], trade_log_df['Expiry'].values[0])
    print(itm_long_current_price)

    # pe_short_pnl = trade_log_df['Pe_Short_En_Price'].values[0] - atm_short_current_price
    # pe_otm_long_pnl = otm_long_current_price - trade_log_df['Pe_otm_Long_En_Price'].values[0]
    # pe_itm_long_pnl = itm_long_current_price - trade_log_df['Pe_itm_Long_En_Price'].values[0]

    # total_pnl = pe_short_pnl + pe_otm_long_pnl + pe_itm_long_pnl
    # trade_log_df['Total_PnL'] = total_pnl
    # return total_pnl

calculate_pnl(trade_log_df, options_df_2018, '2018-01-04 09:30:00')


59143    79.0
Name: Close, dtype: float64
59133    0.25
Name: Close, dtype: float64
59151    440.3
Name: Close, dtype: float64


In [ ]:
pnl_data = []

# Get current market price 
def get_current_market_price(spot_df,date,time,strike_price):
    """
    This function gets the current market price of the given ticker.
    """
    current_price = spot_df[(spot_df['Date']==date)&(spot_df['Time']==time) & (spot_df['Close']==strike_price)]['Close']
    return current_price

def get_option_market_price(options_df,date,time,strike_price,expiry_date):
    """
    This function gets the current market price of the given ticker.
    """
    current_price = options_df[(options_df['Date']==date)&(options_df['Time']==time) & (options_df['Strike Price']==strike_price) & (options_df['ExpiryDate']==expiry_date) ]['Close']
    return current_price

# Function to calculate P&L for the trade
def calculate_pnl(trade_log_df,options_df,date,time):
    atm_short_current_price = get_option_market_price(options_df,date,time,trade_log_df['Pe_Short_Strike'],trade_log_df['Expiry'])#filter data for atm_strike_price,otm_strike_price and itm_strike_price and expity
    
    otm_long_current_price = get_option_market_price(options_df,date,time,trade_log_df['Pe_otm_Long_Strike'],trade_log_df['Expiry'])
    itm_long_current_price = get_option_market_price(options_df,date,time,trade_details['Pe_itm_Long_Strike'],trade_log_df['Expiry'])
    
    pe_short_pnl = trade_log_df['Pe_Short_En_Price'] - atm_short_current_price
    pe_otm_long_pnl = otm_long_current_price - trade_log_df['Pe_otm_Long_En_Price']
    pe_itm_long_pnl = itm_long_current_price - trade_log_df['Pe_itm_Long_En_Price']
    
    total_pnl = pe_short_pnl + pe_otm_long_pnl + pe_itm_long_pnl
    trade_log_df['Total_PnL'] = total_pnl
    return total_pnl

for _,row in options_df_2018.iterrows():
    total_pnl = calculate_pnl(trade_log_df, options_df_2018, row['Date'], row['Time']).values[0]

    
    # Check if target or stop-loss is hit
    if trade_log_df['Total_PnL'] >= trade_log_df['Target']:
        trade_log_df['Exit_Reason'] = 'Target Hit'
        break
    elif trade_log_df['Total_PnL'] <= trade_log_df['Stop_Loss']:
        trade_log_df['Exit_Reason'] = 'Stop Loss Hit'
        break
    elif time == '15:19:59':
        trade_log_df['Exit_Reason'] = 'Held till EOD'
        break

# Save and display the final result
trade_details['Exit Time'] = datetime.now()

pnl_df = pd.DataFrame(pnl_data)



In [333]:
def monitor_and_exit_trades(trade_log_df, options_df):
    """
    This function monitors the trades in the trade log and exits them based on the specified conditions:
    1. A 30% stop loss based on the premium collected from selling the straddle.
    2. An 80% target.
    3. If neither condition is met, hold the position until 15:19:59 on the expiry date and exit all legs of the trade together.

    Parameters:
    trade_log_df (pd.DataFrame): DataFrame containing the trade log entries
    options_df (pd.DataFrame): DataFrame containing options data

    Returns:
    pd.DataFrame: Updated trade log with exit prices and times
    """

    

    # Get current market price 
    def get_current_market_price(spot_df,date,time,strike_price):
        """
        This function gets the current market price of the given ticker.
        """
        current_price = spot_df[(spot_df['Date']==date)&(spot_df['Time']==time) & (spot_df['Close']==strike_price)]['Close']
        return current_price
    
    def get_option_market_price(options_df,datetime,strike_price,expiry_date):
        """
        This function gets the current market price of the given ticker.
        """
        df = options_df[(options_df['Datetime']==datetime) & (options_df['Strike Price']==strike_price) & (options_df['ExpiryDate']==expiry_date) ]
        current_price = df['Close']
        current_strike_price = df['Strike Price']
        return current_price,current_strike_price
    
    def calculate_pnl(trade_log_df, options_df, datetime):
        atm_short_current_price = get_option_market_price(options_df_2018, datetime, trade_log_df['Pe_Short_Strike'].values[0], trade_log_df['Expiry'].values[0])
        print(atm_short_current_price)
        otm_long_current_price = get_option_market_price(options_df_2018, datetime, trade_log_df['Pe_otm_Long_Strike'].values[0], trade_log_df['Expiry'].values[0])
        print(otm_long_current_price)
        itm_long_current_price = get_option_market_price(options_df_2018, datetime, trade_log_df['Pe_itm_Long_Strike'].values[0], trade_log_df['Expiry'].values[0])
        print(itm_long_current_price)

        pe_short_pnl = trade_log_df['Pe_Short_En_Price'].values[0] - atm_short_current_price
        pe_otm_long_pnl = otm_long_current_price - trade_log_df['Pe_otm_Long_En_Price'].values[0]
        pe_itm_long_pnl = itm_long_current_price - trade_log_df['Pe_itm_Long_En_Price'].values[0]

        total_pnl = pe_short_pnl + pe_otm_long_pnl + pe_itm_long_pnl
        trade_log_df['Total_PnL'] = total_pnl
        return total_pnl
    
    

    for index, trade in trade_log_df.iterrows():
        entry_datetime = pd.to_datetime(f"{trade['Date']} {trade['Entry_Time']}")
        expiry_datetime = pd.to_datetime(f"{trade['Expiry']} 15:19:59")

        options_contract_data = options_df[(options_df['ExpiryDate'] == trade['Expiry']) & 
                                                    (options_df['Datetime'] >= entry_datetime) & 
                                                    (options_df['Datetime'] <= expiry_datetime)]

        # Initialize stop loss, target, and premium received
        total_premium_received =   trade['Total_Premium_Received']
        stop_loss = trade['Stop_Loss']
        target = trade['Target']
        exit_reason = 'Held till expiry'
        total_pnl = None
        
        pe_short_exit_price = None
        
        pe_long_exit_price = None
        exit_time = None

        for current_datetime, group in options_contract_data.groupby('Datetime'):
            pe_short_close = group[(group['Strike Price'] == trade['Pe_Short_Strike']) & (group['Instrument Type'] == 'PE')]['Close']
            print(pe_short_close)
            pe_otm_long_close = group[(group['Strike Price'] == trade['Pe_otm_Long_Strike']) & (group['Instrument Type'] == 'PE')]['Close']
            print(pe_otm_long_close)
            pe_itm_long_close = group[(group['Strike Price'] == trade['Pe_itm_Long_Strike']) & (group['Instrument Type'] == 'PE')]['Close']
            print(pe_itm_long_close)

            if not (pe_short_close.empty  or pe_otm_long_close.empty or pe_itm_long_close.empty):
                
                pe_short_close = pe_short_close
            
                pe_otm_long_close = pe_otm_long_close

                pe_itm_long_close = pe_itm_long_close

                total_pnl = calculate_pnl(trade,group,current_datetime)

                if total_pnl <= -stop_loss:
                    exit_reason = 'Stop Loss Hit'
                    total_pnl = total_pnl
                    ce_short_exit_price = ce_short_close
                    pe_short_exit_price = pe_short_close
                    ce_long_exit_price = ce_long_close
                    pe_long_exit_price = pe_long_close
                    exit_time = current_datetime
                    break

                if total_pnl >= target:
                    exit_reason = 'Target Achieved'
                    total_pnl = total_pnl
                    ce_short_exit_price = ce_short_close
                    pe_short_exit_price = pe_short_close
                    ce_long_exit_price = ce_long_close
                    pe_long_exit_price = pe_long_close
                    exit_time = current_datetime
                    break

        if exit_reason == 'Held till end of day':
            exit_time = '15:00:00'

            options_expiry_data = options_df[(options_df['Date'] == trade['Date']) &
                                             (options_df['Time'] == exit_time)]
            
            pe_otm_long_exit_price = options_expiry_data[(options_expiry_data['Strike Price'] == trade['Pe_otm_Long_Strike']) & (options_expiry_data['Instrument Type'] == 'PE')]['Close'].values[0]
            pe_short_exit_price = options_expiry_data[(options_expiry_data['Strike Price'] == trade['Pe_Short_Strike']) & (options_expiry_data['Instrument Type'] == 'PE')]['Close'].values[0]
            pe_itm_long_exit_price = options_expiry_data[(options_expiry_data['Strike Price'] == trade['Pe_itm_Long_Strike']) & (options_expiry_data['Instrument Type'] == 'PE')]['Close'].values[0]

            datetime = pd.to_datetime(f"{trade['Date']} {exit_time}")
            total_pnl = calculate_pnl(trade,options_expiry_data,datetime)

        
        trade_log_df.at[index, 'Pe_Ex_Date'] = exit_time
        trade_log_df.at[index, 'Pe_Short_Ex_Price'] = pe_short_exit_price
        trade_log_df.at[index, 'Pe_itm_Long_Ex_Price'] = pe_itm_long_exit_price
        trade_log_df.at[index, 'Pe_otm_Long_Ex_Price'] = pe_otm_long_exit_price
        trade_log_df.at[index, 'Exit_Reason'] = exit_reason
        trade_log_df.at[index, 'Total_PnL'] = total_pnl

    return trade_log_df


In [334]:
monitor_and_exit_trades(trade_log_df, options_df_2018)

16296    129.0
Name: Close, dtype: float64
16286    11.4
Name: Close, dtype: float64
16304    431.1
Name: Close, dtype: float64


AttributeError: 'int' object has no attribute 'values'

In [318]:
trade_log_df.columns

Index(['Serial_No', 'Date', 'Expiry', 'En_Date', 'Pe_En_Date', 'Ce_Ex_Date',
       'Pe_Ex_Date', 'Atm_Strike', 'Pe_Short_Strike', 'Pe_otm_Long_Strike',
       'Pe_itm_Long_Strike', 'Capital', 'Pe_Short_En_Price',
       'Pe_Short_Ex_Price', 'Pe_otm_Long_En_Price', 'Pe_otm_Long_Ex_Price',
       'Pe_itm_Long_En_Price', 'Pe_itm_Long_Ex_Price', 'Entry_Time',
       'Exit Time', 'Current Market Price', 'Ticker_Pe_Short',
       'Ticker_otm_Pe_Long', 'Ticker_itm_Pe_Long', 'Exit_Reason', 'Stop_Loss',
       'Total_Premium_Received', 'Target', 'Total_PnL'],
      dtype='object')

In [ ]:
# can you create a global variable trade_log_df and update it in the function create_straddle by appending trade_log to it
global trade_log_df
trade_log_df = pd.DataFrame()

def create_straddle(options_df_2018, spot_df_2018, selected_date, entry_time, capital):
    """
    This function creates a straddle by taking an entry in options for a specific date using the ATM strike price.
    It also adds 2% away wings for protection by buying both an out-of-the-money call option and an out-of-the-money put option.
    It logs the trades in a trade log.

    Parameters:
    options_df (pd.DataFrame): DataFrame containing options data
    spot_df (pd.DataFrame): DataFrame containing futures data
    selected_date (str): The date for which to create the straddle (in 'YYYY-MM-DD' format)
    entry_time (str): The time at which the trade is entered (in 'HH:MM:SS' format)
    capital (float): The total capital available for trading

    Returns:
    list: Updated trade log
    """

    # Calculate the ATM strike price for the selected date
    atm_strike_price = calculate_atm_strike_price_for_date(spot_df, selected_date, entry_time)
    if atm_strike_price is None:
        print(f"No ATM strike price found for {selected_date} at {entry_time}.")
        return []

    # Ensure 'Date' and 'Time' columns are in datetime format
    options_df_2018['ExpiryDate'] = pd.to_datetime(options_df['ExpiryDate'], format='%d%b%y')

    
    # Filter options data for the specified date and entry time
    entry_time_dt = pd.to_datetime(entry_time).time()
    options_for_date = options_df_2018[options_df_2018['Datetime']==pd.to_datetime(selected_date + ' ' + entry_time)]

    if options_for_date.empty:
        print(f"No options data found for {selected_date} at {entry_time}.")
        return []

    # Find the closest expiry date to the selected date
    closest_expiry = options_for_date['ExpiryDate'].min()

    # Filter options data for the closest expiry date
    options_for_closest_expiry = options_for_date[options_for_date['ExpiryDate'] == closest_expiry]

    # Calculate the 2% away strikes and round them to the nearest 100 points
    strike_increment = atm_strike_price * 0.02
    itm_put_strike = round((atm_strike_price + strike_increment) / 100) * 100
    otm_put_strike = round((atm_strike_price - strike_increment) / 100) * 100

    print(f"ITM put strike:{itm_put_strike}")
    print(f"OTM put strike:{otm_put_strike}")
    
    # Separate call and put options for the ATM and OTM strikes
    put_options_atm = options_for_closest_expiry[(options_for_closest_expiry['Instrument Type'] == 'PE') &
                                                 (options_for_closest_expiry['Strike Price'] == atm_strike_price)]
    
    put_options_otm = options_for_closest_expiry[(options_for_closest_expiry['Instrument Type'] == 'PE') &
                                                 (options_for_closest_expiry['Strike Price'] == otm_put_strike)]

    put_options_itm = options_for_closest_expiry[(options_for_closest_expiry['Instrument Type'] == 'PE') &
                                                 (options_for_closest_expiry['Strike Price'] == itm_put_strike)]
    
    flag = 0
    lot_size = 15
    lot_count = 1
    

    if put_options_atm.empty :
        print(f"Atm strike price of  {atm_strike_price} on {selected_date} at {entry_time} not FOUND.")
        return []
    
    if put_options_otm.empty:
        print(f"OTM strike price of  {otm_put_strike} on {selected_date} at {entry_time} not FOUND. Finding next closest strike price.")
        # Find the closest strike price to the calculated OTM put strike
        put_options_otm = options_for_closest_expiry[(options_for_closest_expiry['Strike Price']>otm_put_strike) & (options_for_closest_expiry['Strike Price']<atm_strike_price)]
        print(f"The next closest strike price is:{put_options_otm['Strike Price'].iloc[0]}")
        flag=0

    if put_options_itm.empty:
        print(f"ITM strike price of  {itm_put_strike} on {selected_date} at {entry_time} not FOUND.Finding next closest strike price")
        # Find the closest strike price to the calculated ITM put strike
        put_options_itm = options_for_closest_expiry[(options_for_closest_expiry['Strike Price']<itm_put_strike) & (options_for_closest_expiry['Strike Price']>atm_strike_price)]
        print(f"The next closest strike price is:{put_options_itm['Strike Price'].iloc[-1]}")
        flag=1
       
    
    

        
    if flag==1:
        atm_entry_price = put_options_atm.iloc[0]['Close']
        itm_entry_price = put_options_itm.iloc[-1]['Close']
        otm_entry_price = put_options_otm.iloc[0]['Close']
        total_entry_premium = (2*atm_entry_price - itm_entry_price - otm_entry_price)
        print(f"Entry premium for one lot :{total_entry_premium}")
        total_entry_premium_lot = (total_entry_premium)*(lot_size)*(lot_count)
        print(f"Total entry premium for {lot_count} lots :{total_entry_premium_lot}")
        max_profit =  2*atm_entry_price - otm_entry_price +(itm_entry_price-(put_options_itm.iloc[-1]['Strike Price']-atm_strike_price))
        max_profit = max_profit*lot_count*lot_size
        print(f"Max profit in rs.:{max_profit}")
        
        max_loss_1 = (2*atm_entry_price - otm_entry_price  - itm_entry_price)*lot_size
        max_loss_2 =( -otm_entry_price - (atm_strike_price - put_options_otm.iloc[0]['Strike Price'])*2 + (put_options_itm.iloc[-1]['Strike Price']-put_options_atm.iloc[0]['Strike Price']))*lot_size
        if max_loss_1>max_loss_2:
            max_loss = max_loss_1
        else:
            max_loss = max_loss_2
        print(f"Max loss for one lot :{max_loss}")
        max_loss= max_loss*lot_count
        print(f"Max loss in Rs. :{max_loss}")

        upper_breakeven = itm_entry_price -total_entry_premium
        lower_breakeven = otm_entry_price + total_entry_premium
        print(f"Upper breakeven is :{upper_breakeven}")
        print(f"Lower breakeven is :{lower_breakeven}")
        target = (max_profit*0.7)
        stop_loss = (max_loss*0.5)
         # Initialize trade log
        trade_log = []

        # Log the trades
        trade_log = put_log_trades(trade_log, selected_date, closest_expiry, entry_time, atm_strike_price, put_options_atm.iloc[0], put_options_otm.iloc[0],put_options_itm.iloc[-1], capital, stop_loss,total_entry_premium_lot, target)
    
    else:
        atm_entry_price = put_options_atm.iloc[0]['Close']
        itm_entry_price = put_options_itm.iloc[0]['Close']
        otm_entry_price = put_options_otm.iloc[0]['Close']
        total_entry_premium = (2*atm_entry_price - itm_entry_price - otm_entry_price)
        print(f"Entry premium for one lot :{total_entry_premium}")
        total_entry_premium_lot = (total_entry_premium)*(lot_size)*(lot_count)
        print(f"Total entry premium for {lot_count} lots :{total_entry_premium_lot}")
        max_profit = 2*atm_entry_price - otm_entry_price +(itm_entry_price-(put_options_itm.iloc[0]['Strike Price']-atm_strike_price))
        max_profit = max_profit*lot_count*lot_size
        print(f"Max profit is :{max_profit}")
        max_loss_1 = (2*atm_entry_price - otm_entry_price  - itm_entry_price)*lot_size
        max_loss_2 =( -otm_entry_price - (atm_strike_price - put_options_otm.iloc[0]['Strike Price'])*2 + (put_options_itm.iloc[0]['Strike Price']-put_options_atm.iloc[0]['Strike Price']))*lot_size
        if max_loss_1>max_loss_2:
            max_loss = max_loss_1
        else:
            max_loss = max_loss_2
        max_loss = max_loss*lot_count
        print(f"Max loss in rs:{max_loss}")
        upper_breakeven = itm_entry_price -total_entry_premium
        lower_breakeven = otm_entry_price + total_entry_premium
        print(f"Upper breakeven is :{upper_breakeven}")
        print(f"Lower breakeven is :{lower_breakeven}")
        stop_loss = (max_loss*0.5)
        target = (max_profit*0.7)
         # Initialize trade log
        trade_log = []

        # Log the trades
        trade_log = put_log_trades(trade_log, selected_date, closest_expiry, entry_time,exit_time,current_market_price, atm_strike_price, put_options_atm.iloc[0], put_options_otm.iloc[0],put_options_itm.iloc[0], capital, stop_loss,total_entry_premium_lot, target)

    # Update the global trade log
    global trade_log_df
    trade_log_df = pd.concat([trade_log_df, pd.DataFrame(trade_log)])
    print(f"Trade log df is :{trade_log_df}")

    print(f"Current trade log is :{trade_log}")
   
    
    return trade_log


In [341]:
create_straddle(options_df_2018, spot_df_2018, '2018-01-08', '09:30:00', 1000000)

ITM put strike:26200
OTM put strike:25200
ITM strike price of  26200 on 2018-01-08 at 09:30:00 not FOUND.Finding next closest strike price
The next closest strike price is:26000
Entry premium for one lot :-99.40000000000002
Total entry premium for 1 lots :-1491.0000000000002
Max profit in rs.:3409.5
Max loss for one lot :-1491.0000000000005
Max loss in Rs. :-1491.0000000000005
Trade log df is :   Serial_No        Date     Expiry     En_Date  Pe_En_Date Ce_Ex_Date  \
0          1  2018-01-02 2018-01-04  2018-01-02  2018-01-02       None   
0          1  2018-01-08 2018-01-11  2018-01-08  2018-01-08       None   
0          1  2018-01-01 2018-01-04  2018-01-01  2018-01-01       None   
0          1  2018-01-02 2018-01-04  2018-01-02  2018-01-02       None   
0          1  2018-01-03 2018-01-04  2018-01-03  2018-01-03       None   
0          1  2018-01-04 2018-01-04  2018-01-04  2018-01-04       None   
0          1  2018-01-05 2018-01-11  2018-01-05  2018-01-05       None   
0          

C:\Users\ALGO1\AppData\Local\Temp\2\ipykernel_16156\3253925535.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  options_df_2018['ExpiryDate'] = pd.to_datetime(options_df['ExpiryDate'], format='%d%b%y')


[{'Serial_No': 1,
  'Date': '2018-01-08',
  'Expiry': Timestamp('2018-01-11 00:00:00'),
  'En_Date': '2018-01-08',
  'Pe_En_Date': '2018-01-08',
  'Ce_Ex_Date': None,
  'Pe_Ex_Date': None,
  'Atm_Strike': 25700,
  'Pe_Short_Strike': 25700,
  'Pe_otm_Long_Strike': 25200,
  'Pe_itm_Long_Strike': 26000,
  'Capital': 1000000,
  'Pe_Short_En_Price': 112.25,
  'Pe_Short_Ex_Price': None,
  'Pe_otm_Long_En_Price': 10.55,
  'Pe_otm_Long_Ex_Price': None,
  'Pe_itm_Long_En_Price': 313.35,
  'Pe_itm_Long_Ex_Price': None,
  'Entry_Time': '09:30:00',
  'Exit Time': None,
  'Current Market Price': None,
  'Ticker_Pe_Short': 'BANKNIFTY',
  'Ticker_otm_Pe_Long': 'BANKNIFTY',
  'Ticker_itm_Pe_Long': 'BANKNIFTY',
  'Exit_Reason': None,
  'Stop_Loss': -745.5000000000002,
  'Total_Premium_Received': -1491.0000000000002,
  'Target': 2386.6499999999996,
  'Total_PnL': None}]

In [343]:
trade_log

[{'Serial_No': 1,
  'Date': '2018-12-31',
  'Expiry': Timestamp('2019-01-03 00:00:00'),
  'En_Date': '2018-12-31',
  'Pe_En_Date': '2018-12-31',
  'Ce_Ex_Date': None,
  'Pe_Ex_Date': None,
  'Atm_Strike': 27200,
  'Pe_Short_Strike': 27200,
  'Pe_otm_Long_Strike': 26700,
  'Pe_itm_Long_Strike': 27700,
  'Capital': 1000000,
  'Pe_Short_En_Price': 165.0,
  'Pe_Short_Ex_Price': None,
  'Pe_otm_Long_En_Price': 30.0,
  'Pe_otm_Long_Ex_Price': None,
  'Pe_itm_Long_En_Price': 510.0,
  'Pe_itm_Long_Ex_Price': None,
  'Entry_Time': '09:30:00',
  'Ticker_Pe_Short': 'BANKNIFTY',
  'Ticker_Pe_Long': 'BANKNIFTY',
  'Exit_Reason': None,
  'Stop_Loss': 10000.0,
  'Total_Premium_Received': -210.0,
  'Target': 315.0,
  'Total_PnL': None}]

In [ ]:
spot_df_2018.head(40)

In [ ]:
def calculate_performance_stats(trade_log_df):
    """
    Calculate performance statistics from the trade log.

    Parameters:
    trade_log_df (pd.DataFrame): DataFrame containing the trade log entries

    Returns:
    dict: Dictionary containing performance statistics
    """
    total_trades = len(trade_log_df)
    total_profit_loss = trade_log_df['Total_PnL'].sum()
    average_profit_loss = trade_log_df['Total_PnL'].mean()
    win_rate = (trade_log_df[trade_log_df['Total_PnL'] > 0].count()['Total_PnL'] / total_trades) * 100
    loss_rate = (trade_log_df[trade_log_df['Total_PnL'] < 0].count()['Total_PnL'] / total_trades) * 100
    max_drawdown = trade_log_df['Total_PnL'].min()
    max_profit = trade_log_df['Total_PnL'].max()
    
    return {
        'Total Trades': total_trades,
        'Total Profit/Loss': total_profit_loss,
        'Average Profit/Loss per Trade': average_profit_loss,
        'Win Rate (%)': win_rate,
        'Loss Rate (%)': loss_rate,
        'Max Drawdown': max_drawdown,
        'Max Profit': max_profit
    }


In [ ]:
def get_unique_dates(spot_df):
    """
    This function retrieves unique dates from the futures DataFrame.

    Parameters:
    spot_df (pd.DataFrame): Input DataFrame with 'Date' column

    Returns:
    pd.DataFrame: A DataFrame containing unique dates in the DataFrame
    """
    # Ensure 'Date' column is in datetime format
    spot_df['Date'] = pd.to_datetime(spot_df['Date'], format='%Y-%m-%d')

    # Retrieve unique dates
    unique_dates = spot_df['Date'].dt.date.unique()

    # Create a DataFrame with the unique dates
    unique_dates_df = pd.DataFrame(unique_dates, columns=['Date'])

    return unique_dates_df


In [ ]:
def plot_equity_curve(trade_log_df):
    """
    Plot the equity curve and drawdown from the trade log.

    Parameters:
    trade_log_df (pd.DataFrame): DataFrame containing the trade log entries
    """
    # Calculate cumulative profit/loss
    trade_log_df['Cumulative_PnL'] = trade_log_df['Total_PnL'].cumsum()
    
    # Calculate drawdown
    trade_log_df['Drawdown'] = trade_log_df['Cumulative_PnL'] - trade_log_df['Cumulative_PnL'].cummax()
    
    # Plot equity curve
    plt.figure(figsize=(12, 6))
    plt.plot(trade_log_df['Date'], trade_log_df['Cumulative_PnL'], label='Equity Curve')
    plt.plot(trade_log_df['Date'], trade_log_df['Drawdown'], label='Drawdown', color='red')
    plt.xlabel('Date')
    plt.ylabel('PnL')
    plt.title('Equity Curve and Drawdown')
    plt.legend()
    plt.grid(True)
    plt.savefig('equity_curve.png')
    plt.show()

In [ ]:
trade_log_df = pd.DataFrame(trade_log)
trade_log_df

In [ ]:


def main():
    # options_df = pd.read_csv('Banknifty_Options_2018-July_2023.csv')
    # spot_df = pd.read_csv('Banknifty_Spot_2018-2023.csv')
    # # Ensure 'Date' and 'Time' columns are in datetime format
    # # Data cleaning and preprocessing
    # spot_df['Date'] = pd.to_datetime(spot_df['Date'], format='%Y-%m-%d')
    # spot_df['Time'] = pd.to_datetime(spot_df['Time'], format='%H:%M:%S').dt.time
    # options_df['Instrument Type'] = options_df['Strike Price'].str[-1]
    # options_df['Strike Price'] = options_df['Strike Price'].str[:-1].astype(int)
    # options_df['Instrument Type'] = options_df['Instrument Type'].apply(lambda x: x + 'E')
    # options_df['Date'] = pd.to_datetime(options_df['Date'], format='%Y-%m-%d')
    # options_df['ExpiryDate'] = pd.to_datetime(options_df['ExpiryDate'], format='%d%b%y')
    # options_df['Datetime'] = pd.to_datetime(options_df['Date'].astype(str) + ' ' + options_df['Time'].astype(str))
    # spot_df['Datetime'] = pd.to_datetime(spot_df['Date'].astype(str) + ' ' + spot_df['Time'].astype(str))
    # spot_df['Datetime'] = spot_df['Datetime'] + pd.DateOffset(seconds=1)


    # Get unique dates from futures_df
    unique_dates_df = get_unique_dates(spot_df_2018)

    # Print the DataFrame containing unique dates
    print("Unique Dates DataFrame:")
    print(unique_dates_df)

    # Define trading parameters
    entry_time = '09:30:00'
    total_capital = 100000  # Example capital amount

    # Initialize a DataFrame to collect all trade logs
    all_trade_logs = []
    error_log = []

    for _, row in unique_dates_df.iterrows():
        selected_date = row['Date']
        
    
        print(f"Processing date: {selected_date}")

        try:
            # Create straddle and log trades
            trade_log = create_straddle(options_df_2018, spot_df_2018, str(selected_date), entry_time, total_capital)

            print(trade_log)
           
            
                # error_log.append({'Date': selected_date, 'Error': 'No trades were logged'})
        except Exception as e:
            error_log.append({'Date': selected_date, 'Error': str(e)})
            print(f"Error processing date {selected_date}: {e}")

   
    updated_trade_log_df = monitor_and_exit_trades(trade_log_df, options_df_2018)

    if all_trade_logs:
        final_trade_log_df = pd.concat(all_trade_logs,ignore_index=True)
        final_trade_log_df.to_csv('final_trades_log.csv', index=False)
        print("Final trade log has been saved to final_trade_log.csv")
        
        # Calculate performance statistics
        performance_stats = calculate_performance_stats(final_trade_log_df)
        print("Performance Statistics:")
        for key, value in performance_stats.items():
            print(f"{key}: {value}")

        # Plot equity curve and drawdown
        plot_equity_curve(final_trade_log_df)
    else:
        print("No trades were logged.")

    if error_log:
        error_log_df = pd.DataFrame(error_log)
        error_log_df.to_csv('error_logs.csv', index=False)
        print("Error log has been saved to error_log.csv")
    else:
        print("No errors encountered.")

if __name__ == "__main__":
    main()

In [ ]:
options_df_2018[(options_df_2018["Instrument Type"]=="PE") & (options_df_2018["Datetime"]=="2018-01-01 09:30:00")]


In [ ]:
trade_log

In [ ]:
# Filter options_df_2018 to only january 2018 do the same for spot _df
options_df_2018 = options_df_2018[(options_df_2018['Date'].dt.year==2018)&(options_df_2018['Date'].dt.month == 1)]
spot_df_2018 = spot_df_2018[(spot_df_2018['Date'].dt.year==2018)&(spot_df_2018['Date'].dt.month == 1)]

In [ ]:
options_df_2018

In [ ]:
spot_df_2018

In [ ]:
options_df_2018['Date'] = pd.to_datetime(options_df_2018['Date'], format='%Y-%m-%d')
options_df_2018['ExpiryDate'] = pd.to_datetime(options_df_2018['ExpiryDate'], format='%d%b%y')
options_df_2018['Datetime'] = pd.to_datetime(options_df_2018['Date'].astype(str) + ' ' + options_df_2018['Time'].astype(str))
spot_df_2018['Datetime'] = pd.to_datetime(spot_df_2018['Date'].astype(str) + ' ' + spot_df_2018['Time'].astype(str))
options_df_2018['Datetime'] = options_df_2018['Datetime'] + pd.DateOffset(seconds=1)

In [358]:
import pandas as pd
from datetime import datetime, time

def get_option_market_price(pe_options_df_2018, current_datetime, strike_price, expiry_date):
    """
    Fetch the current market price for the specified option criteria.
    """
    df = pe_options_df_2018[(pe_options_df_2018['Datetime'] == current_datetime) & 
                    (pe_options_df_2018['Strike Price'] == strike_price) & 
                    (pe_options_df_2018['Expiry Date'] == expiry_date)]
    if not df.empty:
        current_price = df.iloc[-1]['Close']  # Get the most recent closing price
        return current_price
    else:
        return None

def calculate_pnl(trade_log_df, pe_options_df_2018, current_datetime):
    """
    Calculate the P&L for the trade at the given datetime.
    """
    atm_short_current_price = get_option_market_price(pe_options_df_2018, current_datetime, trade_log_df['Pe_Short_Strike'], trade_log_df['Expiry'])
    otm_long_current_price = get_option_market_price(pe_options_df_2018, current_datetime, trade_log_df['Pe_otm_Long_Strike'], trade_log_df['Expiry'])
    itm_long_current_price = get_option_market_price(pe_options_df_2018, current_datetime, trade_log_df['Pe_itm_Long_Strike'], trade_log_df['Expiry'])

    if None in [atm_short_current_price, otm_long_current_price, itm_long_current_price]:
        return None  # If any prices are not found, return None

    pe_short_pnl = trade_log_df['Pe_Short_En_Price'] - atm_short_current_price
    pe_otm_long_pnl = otm_long_current_price - trade_log_df['Pe_otm_Long_En_Price']
    pe_itm_long_pnl = itm_long_current_price - trade_log_df['Pe_itm_Long_En_Price']

    total_pnl = pe_short_pnl + pe_otm_long_pnl + pe_itm_long_pnl
    trade_log_df['Total_PnL'] = total_pnl
    return total_pnl

def monitor_trades(trade_log_df, pe_options_df_2018, start_datetime, end_time=time(15, 15, 59)):
    """
    Monitor and update the trade log dataframe until the trade hits the target, stoploss or the end of day.
    """
    current_time = start_datetime
    while current_time.time() <= end_time:
        pnl = calculate_pnl(trade_log_df, options_df, current_time)
        if pnl is None:
            print("Market data not available for", current_time)
        else:
            print("Current PnL:", pnl)
            if pnl <= trade_log_df['Stop_Loss']:
                print("Trade stopped due to hitting stop loss.")
                trade_log_df['Exit Time'] = current_time
                trade_log_df['Exit_Reason'] = 'Stop Loss Hit'
                break
            elif pnl >= trade_log_df['Target']:
                print("Trade stopped due to hitting target.")
                trade_log_df['Exit Time'] = current_time
                trade_log_df['Exit_Reason'] = 'Target Hit'
                break
        current_time += pd.Timedelta(minutes=1)  # Increment time by 1 minute

    # Ensuring the 'Exit Time' is recorded if no stop or target is hit by the end of the day
    if 'Exit Time' not in trade_log_df or trade_log_df['Exit Time'].isna().all():
        trade_log_df['Exit Time'] = end_time
        trade_log_df['Exit_Reason'] = 'End of Day'

    print("Monitoring ended.")

# Example usage
# trade_log_df should be a DataFrame containing the details of the trade
# options_df should be a DataFrame containing the options market data
# start_datetime should be a datetime.datetime object representing when to start monitoring


In [359]:
import pandas as pd

# Define the structure of the DataFrame based on the given output example
columns = [
    'Date', 'Expiry', 'En_Date', 'Pe_En_Date', 'Ce_Ex_Date', 'Pe_Ex_Date',
    'Atm_Strike', 'Pe_Short_Strike', 'Pe_otm_Long_Strike', 'Pe_itm_Long_Strike',
    'Capital', 'Pe_Short_En_Price', 'Pe_Short_Ex_Price', 'Pe_otm_Long_En_Price',
    'Pe_otm_Long_Ex_Price', 'Pe_itm_Long_En_Price', 'Pe_itm_Long_Ex_Price',
    'Entry_Time', 'Exit Time', 'Current Market Price', 'Ticker_Pe_Short',
    'Ticker_otm_Pe_Long', 'Ticker_itm_Pe_Long', 'Exit_Reason', 'Stop_Loss',
    'Total_Premium_Received', 'Target', 'Total_PnL'
]
results_df = pd.DataFrame(columns=columns)


In [352]:
# Filter for January 2018
options_df_2018 = options_df[(options_df['Date'].dt.year == 2018) & (options_df['Date'].dt.month == 1)].copy()
spot_df_2018 = spot_df[(spot_df['Date'].dt.year == 2018) & (spot_df['Date'].dt.month == 1)].copy()

print(options_df_2018.head())
print(spot_df_2018.head())


        Date     Ticker  Strike Price ExpiryDate      Time    Open    High  \
0 2018-01-01  BANKNIFTY         24800    04JAN18  09:15:59    6.50    6.95   
1 2018-01-01  BANKNIFTY         24900    04JAN18  09:15:59    7.00    9.00   
2 2018-01-01  BANKNIFTY         25000    04JAN18  09:15:59   12.50   13.50   
3 2018-01-01  BANKNIFTY         25100    04JAN18  09:15:59   18.55   20.80   
4 2018-01-01  BANKNIFTY         25200    04JAN18  09:15:59  314.25  314.25   

     Low   Close  Volume  Open Interest Instrument Type  
0   5.95    6.10    4600          70040              PE  
1   6.65    8.30    7480         138320              PE  
2   7.35   12.75   20520         460320              PE  
3   16.9   18.80    9480         147800              PE  
4  314.2  314.20      80          13840              CE  
   Unnamed: 0     Ticker       Date      Time     Open     High      Low  \
0       30610  BANKNIFTY 2018-01-01  09:08:00  25565.8  25565.8  25565.8   
1       30611  BANKNIFTY 2018-0

In [353]:
# Filter for PE instruments
pe_options_df_2018 = options_df_2018[options_df_2018["Instrument Type"] == "PE"].copy()

print(pe_options_df_2018.head())


        Date     Ticker  Strike Price ExpiryDate      Time   Open   High  \
0 2018-01-01  BANKNIFTY         24800    04JAN18  09:15:59   6.50   6.95   
1 2018-01-01  BANKNIFTY         24900    04JAN18  09:15:59   7.00   9.00   
2 2018-01-01  BANKNIFTY         25000    04JAN18  09:15:59  12.50  13.50   
3 2018-01-01  BANKNIFTY         25100    04JAN18  09:15:59  18.55  20.80   
5 2018-01-01  BANKNIFTY         25200    04JAN18  09:15:59  29.90  35.35   

    Low  Close  Volume  Open Interest Instrument Type  
0  5.95   6.10    4600          70040              PE  
1  6.65   8.30    7480         138320              PE  
2  7.35  12.75   20520         460320              PE  
3  16.9  18.80    9480         147800              PE  
5  27.2  30.45   28280         264760              PE  


In [364]:
if 'Time' in options_df.columns:
    
    pe_options_df_2018['Datetime'] = pe_options_df_2018.apply(lambda row: pd.Timestamp.combine(row['Date'], row['Time']), axis=1)

# Ensure spot_df has a 'Datetime' column too
if 'Time' in spot_df.columns:
    
    spot_df_2018['Datetime'] = spot_df_2018.apply(lambda row: pd.Timestamp.combine(row['Date'], row['Time']), axis=1)


In [373]:
pe_options_df_2018['Spot Price']= pe_options_df_2018['Strike Price']

In [374]:
pe_options_df_2018.head()

,Date,Ticker,Strike Price,ExpiryDate,Time,Open,High,Low,Close,Volume,Open Interest,Instrument Type,Datetime,Spot Price
0,2018-01-01,BANKNIFTY,24800,2018-01-04,09:15:59,6.50,6.95,5.95,6.10,4600,70040,PE,2018-01-01 09:15:59,24800
1,2018-01-01,BANKNIFTY,24900,2018-01-04,09:15:59,7.00,9.00,6.65,8.30,7480,138320,PE,2018-01-01 09:15:59,24900
2,2018-01-01,BANKNIFTY,25000,2018-01-04,09:15:59,12.50,13.50,7.35,12.75,20520,460320,PE,2018-01-01 09:15:59,25000
3,2018-01-01,BANKNIFTY,25100,2018-01-04,09:15:59,18.55,20.80,16.9,18.80,9480,147800,PE,2018-01-01 09:15:59,25100
5,2018-01-01,BANKNIFTY,25200,2018-01-04,09:15:59,29.90,35.35,27.2,30.45,28280,264760,PE,2018-01-01 09:15:59,25200


In [371]:
def automate_trading(pe_options_df_2018, spot_df_2018, initial_capital):
    # Assuming 'Datetime' is a datetime column in options_df
    unique_days = pe_options_df_2018['Datetime'].dt.date.unique()

    for day in unique_days:
        print("Processing trades for:", day)
        trade_data = create_straddle(pe_options_df_2018, spot_df_2018, str(day), '09:30:00', initial_capital)
        results_df.loc[len(results_df)] = trade_data
    
    return results_df


print(results_df)


Empty DataFrame
Columns: [Date, Expiry, En_Date, Pe_En_Date, Ce_Ex_Date, Pe_Ex_Date, Atm_Strike, Pe_Short_Strike, Pe_otm_Long_Strike, Pe_itm_Long_Strike, Capital, Pe_Short_En_Price, Pe_Short_Ex_Price, Pe_otm_Long_En_Price, Pe_otm_Long_Ex_Price, Pe_itm_Long_En_Price, Pe_itm_Long_Ex_Price, Entry_Time, Exit Time, Current Market Price, Ticker_Pe_Short, Ticker_otm_Pe_Long, Ticker_itm_Pe_Long, Exit_Reason, Stop_Loss, Total_Premium_Received, Target, Total_PnL]
Index: []

[0 rows x 28 columns]


In [376]:
final_results_df = automate_trading(pe_options_df_2018, spot_df_2018, 1000000)
print(final_results_df)

Processing trades for: 2018-01-01


KeyError: 'Spot Price'

In [369]:
import pandas as pd
from datetime import datetime

def calculate_atm_strike_price_for_date(spot_df, selected_date, entry_time):
    """
    Calculate the ATM strike price for the selected date and time from spot data.
    """
    datetime_str = f"{selected_date} {entry_time}"
    entry_datetime = pd.to_datetime(datetime_str)
    # Assuming spot_df is sorted by Datetime
    closest_spot_record = spot_df_2018[spot_df['Datetime'] <= entry_datetime].iloc[-1]
    return closest_spot_record['Spot Price']

def put_log_trades(selected_date, closest_expiry, entry_time, atm_strike_price, put_options_atm, put_options_otm, put_options_itm, capital, stop_loss, total_premium, target):
    """
    Log the trades into a structured format.
    """
    trades = {
        'Date': selected_date,
        'Expiry': closest_expiry,
        'Entry_Time': entry_time,
        'Atm_Strike': atm_strike_price,
        'Pe_Short_Strike': atm_strike_price,  # Assuming this is the same
        'Pe_otm_Long_Strike': put_options_otm['Strike Price'],
        'Pe_itm_Long_Strike': put_options_itm['Strike Price'],
        'Capital': capital,
        'Stop_Loss': stop_loss,
        'Total_Premium_Received': total_premium,
        'Target': target
    }
    return trades

def create_straddle(options_df, spot_df, selected_date, entry_time, capital):
    """
    Creates a straddle trading position.
    """
    atm_strike_price = calculate_atm_strike_price_for_date(spot_df, selected_date, entry_time)
    if atm_strike_price is None:
        print(f"No ATM strike price found for {selected_date} at {entry_time}.")
        return pd.DataFrame()

    pe_options_df_2018['ExpiryDate'] = pd.to_datetime(pe_options_df_2018['ExpiryDate'], format='%d%b%y').copy()

    # Ensure that options data are available for the selected date
    entry_datetime = pd.to_datetime(f"{selected_date} {entry_time}")
    options_for_date = options_df[(options_df['Datetime'] == entry_datetime)].copy()

    if options_for_date.empty:
        print(f"No options data found for {selected_date} at {entry_time}.")
        return pd.DataFrame()

    # Find the closest expiry
    closest_expiry = options_for_date['ExpiryDate'].min()
    options_for_closest_expiry = options_for_date[options_for_date['ExpiryDate'] == closest_expiry]

    # Calculate 2% away strikes
    strike_increment = atm_strike_price * 0.02
    itm_put_strike = round((atm_strike_price + strike_increment) / 100) * 100
    otm_put_strike = round((atm_strike_price - strike_increment) / 100) * 100

    # Separate options
    put_options_atm = options_for_closest_expiry[(options_for_closest_expiry['Instrument Type'] == 'PE') & (options_for_closest_expiry['Strike Price'] == atm_strike_price)]
    put_options_otm = options_for_closest_expiry[(options_for_closest_expiry['Instrument Type'] == 'PE') & (options_for_closest_expiry['Strike Price'] == otm_put_strike)]
    put_options_itm = options_for_closest_expiry[(options_for_closest_expiry['Instrument Type'] == 'PE') & (options_for_closest_expiry['Strike Price'] == itm_put_strike)]

    if put_options_atm.empty or put_options_otm.empty or put_options_itm.empty:
        print("One or more required options are missing from the dataset.")
        return pd.DataFrame()



    # Logging the trade
    trade_log = put_log_trades(selected_date, closest_expiry, entry_time, atm_strike_price, put_options_atm, put_options_otm, put_options_itm, capital, stop_loss, total_premium, target)
    return pd.DataFrame([trade_log])


